# Tutorial: Exploring a database with Preql

This is a practical guide on how to use Preql to explore an existing database, using the Jupyter environment.

Follow the code examples, and make sure to add your own cells and play around with the code.

## Connect to the database

We tell Preql which database to use, by calling the `connect()` function with a URI.

Since we are connecting for exploration, we will also tell it to load all the tables in the database into the global namespace.

Make sure to adjust the path to the correct one!

In [1]:
uri = "sqlite:///chinook.db"
connect(uri, load_all_tables: true)

If there was no error, it means everything went well.

We can confirm by listing the tables in the database:

In [2]:
tables()

## Connecting Foreign Keys

For now, we will focus on the `tracks` table. We can see in the preview that it contains foreign keys, and common-sense suggests that they relate to the tables `albums` and `genres`. 

In [3]:
tracks

If we wanted, we could join them explicitely with code like 

```javascript
join(t: tracks.AlbumId, a: albums.AlbumId)
```

However, we can make it more convenient, by re-declaring the tables with foreign keys.

We'll manually declare the relevant columns, and use `...` to auto-declare the rest.

In [4]:
table albums {
	ArtistId: artists.ArtistId
	...
}

table tracks {
	AlbumId: albums.AlbumId,
	GenreId: genres.GenreId
	...
}


Now, we can easily join the tables without having to specify on which columns.

(Don't worry, if there was any ambiguity, Preql would throw an error rather than guess.)

In [5]:
join(t: tracks, art: artists, alb: albums, g: genres)

The call to `join()` above, creates four columns of structs, one for each table.

For convenience, we might like to flatten those structs into a single table, and give it a name:

In [6]:
tracks_joined = join(art: artists, alb: albums, t: tracks, g: genres) {
	...t !GenreId !AlbumId !TrackId
	Artist: art.Name
	Album: alb.Title
	Genre: g.Name
}


# Many-to-many

Let's turn out attention to another table, called `playlist_track`.

It serves as a way to connect the `tracks` table with the `playlists` table, which contains the name of the playlist.

In [7]:
playlist_track

Let's write another declaration, to make our future joins simpler:

In [8]:
table playlist_track {
	PlaylistId: playlists.PlaylistId
	TrackId: tracks.TrackId
}

Just for exploration's sake, let's see how many tracks each playlist has.

In [9]:
join(pt: playlist_track, p: playlists) {p.Name => count()}

And let's create a new table of tracks, but this time we'll use the group-by operator (`=>`) to add a list of categories to each track: 

In [10]:
tracks_with_categories = join(t: tracks, pt: playlist_track, p: playlists) {
    ...t => Categories: p.Name
}

## Exercise

Write a query that joins all the following tables, and displays them in a single convenient table:

- tracks
- artists
- albums
- genres
- playlists